In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
%sql sqlite:///my_data1.db

In [5]:
import pandas as pd
df = pd.read_csv("Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

/opt/anaconda3/lib/python3.11/site-packages/pandas/core/generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [6]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


Display the names of the unique launch sites in the space mission

In [8]:
%%sql
SELECT DISTINCT "Launch_Site" FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


Display 5 records where launch sites begin with the string 'CCA'

In [10]:
%%sql
SELECT * 
FROM SPACEXTBL
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5;


 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


Display the total payload mass carried by boosters launched by NASA (CRS)

In [24]:
%%sql
SELECT SUM(PAYLOAD_MASS__KG_) AS TotalPayloadMass
FROM SPACEXTBL
WHERE Customer = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


TotalPayloadMass
45596


Display average payload mass carried by booster version F9 v1.1

In [29]:
%%sql
SELECT AVG(PAYLOAD_MASS__KG_) AS AveragePayloadMass
FROM SPACEXTBL
WHERE Booster_Version = 'F9 v1.1';


 * sqlite:///my_data1.db
Done.


AveragePayloadMass
2928.4


List the date when the first succesful landing outcome in ground pad was acheived.

In [32]:
%%sql
SELECT MIN(Date) AS FirstSuccessfulLandingDate
FROM SPACEXTBL
WHERE Landing_Outcome LIKE '%Success (ground pad)%';

 * sqlite:///my_data1.db
Done.


FirstSuccessfulLandingDate
2015-12-22


List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [41]:
%%sql
SELECT Booster_Version
FROM SPACEXTBL
WHERE Landing_Outcome LIKE '%Success (drone ship)%'
AND PAYLOAD_MASS__KG_ > 4000
AND PAYLOAD_MASS__KG_ < 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


List the total number of successful and failure mission outcomes

In [44]:
%%sql
SELECT Mission_Outcome, COUNT(*) AS TotalCount
FROM SPACEXTBL
GROUP BY Mission_Outcome;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,TotalCount
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


List the names of the booster_versions which have carried the maximum payload mass.

In [51]:
%%sql
SELECT Booster_Version
FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ = (
    SELECT MAX(PAYLOAD_MASS__KG_)
    FROM SPACEXTBL
);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [54]:
%%sql
SELECT STRFTIME('%m', Date) AS Month,
       Landing_Outcome,
       Booster_Version,
       Launch_Site
FROM SPACEXTBL
WHERE STRFTIME('%Y', Date) = '2015'
      AND Landing_Outcome LIKE '%Failure (drone ship)%';


 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [57]:
%%sql
SELECT Landing_Outcome,
       COUNT(*) AS Count
FROM SPACEXTBL
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY Landing_Outcome
ORDER BY Count DESC;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
